## produce a simulated pulse-train from Yuval' simulations
To be then processed by "PETsysEventAnalyzer" similar to the data
in /Users/erezcohen/Desktop/Software/TOFPET2/PETsysAnalysis/CPP

last edit Dec-20, 2021

## definitions

In [65]:
import numpy as np, pandas as pd, matplotlib.pyplot as plt, matplotlib as mpl
import sys; sys.path.insert(0, '/Users/erezcohen/Desktop/Software/mySoftware/Python/'); 
import sys; sys.path.insert(0, '/Users/erezcohen/Desktop/Software/TOFPET2/PETsysAnalysis/Python/'); 
from my_tools               import *; 
from plot_tools             import *;
from my_data_analysis_tools import *;
from PETsys_analysis_tools  import *;

%config InlineBackend.figure_format = 'retina'
plt.rcParams['mathtext.fontset'] = 'stix'
plt.rcParams['font.family'] = 'STIXGeneral'

## auxiliary functions

In [66]:
def merge_pulses(indices,times,window,energies,thresh):
    pulses=dict()
    times=np.array(list(map(float,times)))
    energies=np.array(list(map(float,energies)))
    indices=np.array(list(map(int,indices)))
    
    
    for i in range(N):
        pulses[i]=np.column_stack((times[indices==i], energies[indices==i]))
        pulses[i]=pulses[i][pulses[i][:,0].argsort()]

    
    for key in pulses.keys():
        temp_time=np.array([])
        temp_energy=np.array([])
        det_pulses=pulses[key]
        for i in range(det_pulses.shape[0]):
            if i==0:
                temp_time=np.append(temp_time,det_pulses[0,0])
                temp_energy=np.append(temp_energy,det_pulses[0,1])
            elif (det_pulses[i,0]-temp_time[-1])>window:
                temp_time=np.append(temp_time,det_pulses[i,0])
                temp_energy=np.append(temp_energy,det_pulses[i,1])
            else:
                temp_energy[-1]+=det_pulses[i,1]
        t_e_mat=np.column_stack((temp_time, temp_energy))
        pulses[key]=t_e_mat[t_e_mat[:,1]>thresh]
        
        """
        print(pulses[key])
        print("-----")
    print("-@@-@@-@@-@@-@@-")"""
    return pulses

## select source activity

### units: gr, sec

In [67]:
sec= 1
ms = 1.e-3
ns = 1.e-9
gr = 1
mgr= 1.e-3
ugr= 1.e-6
ngr= 1.e-9
N  = 4 # number of detectors in the BoxSi prototype

In [69]:
Edep_th_gamma    = 0.4 # energy deposition threshold in MeVee
# for 0.4 MeVee, electrons light output is 3e-1. 
# proton light output is 3e-1 for 1.6 MeV energy deposity
# see Verbinsky et al., NIM (1968) or Cecconello, Jour. Fus. Ener. 38 (2019)
Edep_th_neutron  = 1.6
energy_threshold = [Edep_th_gamma,Edep_th_neutron] #Energy Threshold for Gamma detection and for Neutron Detection in MeV

# Since the data has a coincidence window defined, 
# the code that analyses the simulation merges each pair of pulses that the separtion in time between them is smaller than this value
merge_window     = 2.*ns # coincidence time window in [ns]. 

# Cf252 activity to mass
# [https://www.frontier-cf252.com/californium-faqs/]
# 1 µg = 2.31e6 neutrons/second (n/s)
# 1 µg = 0.5mCi
# 1 µg = 0.02 GBq
# source event rate
Cf_activity_uCi  = 4.85
Cf252_uCi_to_gr  = 1e-3 * 1. * ugr/0.5; # 1 µg = 0.5mCi
Cf_mass_gr       = Cf_activity_uCi * Cf252_uCi_to_gr;
uCi_to_Bqrl      = 37.e3; # 1 uCi = 37 kBq
decay_rate_Hz    = Cf_activity_uCi * uCi_to_Bqrl; # 19.85e12*Cf_mass_gr       # decay rate - decays per sec
fisssion_rate_Hz = (3.092/100)*decay_rate_Hz # rate of fission events per second
print('decay rate is',decay_rate_Hz/1e6,'MHz')

decay rate is 0.17945 MHz


## produce pulse train
event files (like e.g. Cf252_10_sec_events.csv) [in /Users/erezcohen/Desktop/data/PETsys/BoxSi_proto2.2/vth_1PE/Cf252_data/]

Record the time in [ms], and include the following structure:

eventID,N(SiPMs),time[ms],Qtot[a.u.],detector,channels

0,51,894.553879236,32.796025,12,[555;548;...;]

In [70]:
in_data_path = "/Users/erezcohen/Desktop/Software/TOFPET2/PETsysAnalysis/Python/BoxSi/proto2.2/Geant4Simulations/YuvalSimulationResults/my_simulations/"
out_data_path = "/Users/erezcohen/Desktop/data/PETsys/BoxSi_proto2.2/Geant4Sims/YuvalSimulations/"

In [71]:
c_names=["Time Gamma","Energy Gamma", "Id Gamma","Time Neutron","Energy Neutron","Id Neutron"]
df = pd.read_csv(in_data_path+'rossi_nt_rossi.csv',sep=',',   header=9, names=c_names)

t_global=0;
particles_names=["Gamma","Neutron"]
det_id=[]
final_pulses_times=np.array([])
final_pulses_energy=np.array([])
final_pulses_id=np.array([])
final_pulses_isN=np.array([])
final_pulses_real_times=np.array([])
eff = 21668/1000000 # efficiency is used for time-ordering, as Geant4 counts only "detected events" and we count here "decay events"

for i in range(df.shape[0]):
    if (i%10000==0):
        print(i)
    while np.random.rand(1)[0]>eff:
        t_global+=np.random.exponential(1/decay_rate_Hz)
    t_global+=np.random.exponential(1/decay_rate_Hz)
    
    for particle_i in range(0,2):
        particle_name=particles_names[particle_i]
        row=df.iloc[i]
        time=row["Time "+particle_name]
        if isinstance(time,str):
            energy=row["Energy "+particle_name].split(";")
            det_index=row["Id "+particle_name].split(";")
            time=row["Time "+particle_name].split(";")
            final_pulses=merge_pulses(det_index,time,merge_window,energy,energy_threshold[particle_i])
            
            for key, item in final_pulses.items():
                if (item.size>0):
                    time_row=item[:,0]+t_global
                    final_pulses_times=np.concatenate([final_pulses_times,time_row])
                    final_pulses_real_times=np.concatenate([final_pulses_real_times,item[:,0]])
                    final_pulses_energy=np.concatenate([final_pulses_energy,item[:,1]])
                    final_pulses_id=np.concatenate([final_pulses_id,key*np.ones(time_row.size)])
                    final_pulses_isN=np.concatenate([final_pulses_isN,(particle_name=="Neutron")*np.ones(time_row.size)])
print('done.')

0
10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
done.


In [72]:
final_pulses_times_sec  = final_pulses_times
final_pulses_times_ns   = final_pulses_times_sec  * 1e9 #to ns
final_pulses_times_ms   = final_pulses_times_sec  * 1e3 #to ns
final_pulses_real_times = final_pulses_real_times * 1e9 
final_pulses_times_ms

array([3.49022569e-01, 5.08601066e-01, 1.26107481e+00, ...,
       2.75914124e+04, 2.75920108e+04, 2.75932009e+04])

In [73]:
print('The simulated measurement of a %.1f uCi 252Cf source took %.1f'%(Cf_activity_uCi,t_global),'Seconds')

The simulated measurement of a 4.8 uCi 252Cf source took 27.6 Seconds


### how to convert simulated detector ID to BoxSi detector ID

In [74]:
def sim_det_id_to_detector(pulse_id):
    detector = (pulse_id+1) * 3
    return detector

## Sort results in a Pandas dataframe a csv file

In [75]:
# eventID,N(SiPMs),time[ms],Qtot[a.u.],detector,channels
titles=["eventID","N(SiPMs)","time[ms]", "Qtot[a.u.]","detector","channel"]
df_final=pd.DataFrame(np.column_stack((range(len(final_pulses_times)),
                                       np.zeros(len(final_pulses_times)),
                                       final_pulses_times/ns*ms,
                                       final_pulses_energy,
                                       sim_det_id_to_detector(final_pulses_id),
                                       np.zeros(len(final_pulses_times)))),
                      columns=titles, index=(range(len(final_pulses_times))))
df_final=df_final.sort_values(by="time[ms]")
df_final = df_final.astype({"eventID": int, "N(SiPMs)": int, 
                            "time[ms]": np.float64, "Qtot[a.u.]": np.float32,
                            "detector": int, "channel": int})
df_final[69:72]

# original from Yuval:
# titles=["Time (ns)","Energy(MeV)","Detector ID", "IsNeutron"]

,eventID,N(SiPMs),time[ms],Qtot[a.u.],detector,channel
69,69,0,48364.000181,0.429046,3,0
70,70,0,48762.919747,0.454656,3,0
71,71,0,48762.919796,0.661901,12,0


## and export to a CSV file

In [76]:
df_final.to_csv(out_data_path + "G4sim_Cf252_%.1fuCi_%.0f_sec_events.csv"%(Cf_activity_uCi,t_global),index=False)
print('done.')
print('saved',out_data_path + "G4sim_Cf252_%.1fuCi_%.0f_sec_events.csv"%(Cf_activity_uCi,t_global))

done.
saved /Users/erezcohen/Desktop/data/PETsys/BoxSi_proto2.2/Geant4Sims/YuvalSimulations/G4sim_Cf252_4.8uCi_28_sec_events.csv
